# Vite (Dev Server)

## ¿Por qué un Dev Server?

Un servidor de desarrollo (Dev Server) es fundamental para mejorar la experiencia de desarrollo. Algunas de las ventajas clave incluyen:

🚀 **Mejora en la Experiencia de Desarrollo**: Proporciona una experiencia de desarrollo fluida y rápida.

🔄 **Carga de Módulos en Tiempo Real**: Permite la recarga de módulos sin necesidad de recargar toda la página.

🌐 **Soporte ESM (ECMAScript Modules)**: Facilita el uso de módulos ECMAScript nativos.

🛠️ **Entorno de Desarrollo Rápido**: Acelera el ciclo de desarrollo con herramientas modernas y eficientes.


Los bundlers, como Vite, agrupan y gestionan múltiples archivos JavaScript, CSS y otros recursos, permitiendo una mejor organización y mantenimiento. Vite utiliza [Rollup](https://rollupjs.org/), en comparación con otra alternativa famosa como [esbuild](https://esbuild.github.io/).

Vite reduce el tamaño total de los archivos mediante técnicas como la minificación y la compresión.

Ofrece soporte para funcionalidades modernas con compatibilidad en navegadores antiguos.

Automatiza tareas como la transpilación de TypeScript a JavaScript, la transpilación de SCSS o SASS a CSS, minificación, ofuscación y la generación de rutas de archivos.

## Configuración

Para iniciar un proyecto con Vite:

1. Crear un nuevo proyecto:
    ```bash
    $ npm create vite@latest my-app -- --template vanilla
    ```

2. Instalar las dependencias:
    ```bash
    $ cd my-app
    $ npm install
    ```

3. Ejecutar el servidor de desarrollo:
    ```bash
    $ npm run dev
    ```

Mover los archivos y las importaciones innecesarias al directorio `./src/` permite mantener una estructura de proyecto limpia y organizada.

## Importación de Bibliotecas

Para importar bibliotecas en Vite, se usa `npm install`:

```bash
$ npm install lodash
```

Luego, en tu archivo JavaScript:

```javascript
import _ from "lodash";
_.join([1, 2]);
```

Vite utiliza una técnica llamada “bare module imports”, que funciona también en TypeScript y Angular, lo cual elimina la necesidad de importar desde `../node_modules…` y de añadir `.js` al final del archivo.

## Importar CSS e Imágenes

Para importar CSS y trabajar con imágenes en Vite:

```javascript
import './style.css';
import javascriptLogo from './javascript.svg';
import viteLogo from '/vite.svg';

const logoHtml = `<img src="${viteLogo}" class="logo" alt="Vite logo" />`;
document.body.innerHTML += logoHtml;
```

El CSS se incluye directamente en el bundle, mientras que las imágenes no se importan directamente, pero sus nombres son modificados en el build. Trabajar con variables permite modificar dinámicamente el nombre de las imágenes.

Vite también permite importar JSON y otros tipos de archivos.

## Vite + Bootstrap

Para usar Bootstrap con Vite, sigue los pasos proporcionados en la [documentación oficial de Bootstrap](https://getbootstrap.com/docs/5.2/getting-started/vite/). Puede que no necesites seguir todos los pasos si ya tienes un proyecto en marcha.

## Build for Production

Para crear una versión de producción del proyecto:

```bash
$ vite build
```

Si necesitas desplegar tu aplicación en una subcarpeta:

```bash
$ vite build --base=/my/public/path/
```